<a href="https://colab.research.google.com/github/Valent0296/allvent.github.io/blob/master/Test_3_tesi_AlexNet_PE2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/MyDrive/math_loader/')

Mounted at /content/drive


In [ ]:
import torch
from torch import Tensor
import torch.nn as nn
from torch.nn import Transformer
from torchvision import datasets, transforms, models
from torchvision.transforms import ToTensor, Lambda
from torch.utils.data import DataLoader
import math
from torch import optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

#####SALVARE IL MODELLO UNA VOLTA ALLENATO, DOPODICHè IN UN ALTRO NOTEBOOK ANDARE AD ANALIZZARE PASSO PER PASSO COSA FA


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = models.alexnet(pretrained=True).features.to(device)
model_ft.add_module("0", nn.Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2)))
model_ft.add_module("12", nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1,1)))
model_ft.add_module("13", nn.AdaptiveAvgPool2d(output_size=(6,6)))
print(model_ft)
for name_ft in model_ft.modules():
  name_ft.requires_grad_(True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:00<00:00, 295MB/s]


Sequential(
  (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): AdaptiveAvgPool2d(output_size=(6, 6))
)


In [ ]:
%cd /content/drive/MyDrive/math_dataloader/
import data as d
import language as l

/content/drive/MyDrive/math_dataloader


In [ ]:
filename = ["data/" + fi for fi in ["train.txt", "test.txt", "validation.txt"]]

training = True
height = 160
width = 1600
dataset=[]
for i in range(3):
  training = True if i == 1 else False
  dataset.append(d.ExprDataset("data/", filename[i], training, height,
                          width, geometric_var=0.05,
                          photometric_var=0.25))
n_tok = len(dataset[0].tokens)

train_dataloader = DataLoader(dataset[0], batch_size=64, shuffle=True)
test_dataloader = DataLoader(dataset[1], batch_size=64, shuffle=True)
val_dataloader = DataLoader(dataset[2], batch_size=64, shuffle=True)

In [ ]:
#######################
# Positional Encoding #
#######################
#1st version trial, future implementations will try diverse function
class PositionalEncoding( nn.Module ):

  def __init__( self, d_model: int=256, dropout: float=0.1, maxlen: int=10 ):
    super(PositionalEncoding, self).__init__()
    den = torch.exp( torch.arange( 0, d_model, 2 ) * ( -math.log(10000) / d_model ) )
    pos = torch.arange(0, maxlen).reshape( maxlen, 1 )
    self.pe = torch.zeros( (maxlen, d_model) ).to(device)
    self.pe[:, 0::2] = torch.sin(pos*den)
    self.pe[:, 1::2] = torch.cos(pos*den)
    self.pe.unsqueeze_(-2) #BE CAREFUL TO THIS ONE SINCE IT CHANGES DIMENSION
    self.dropout = nn.Dropout(dropout)
    self.register_buffer("pos_enc", self.pe)

  def forward(self, ccn_embed: Tensor):
    return self.dropout( ccn_embed + self.pe[ :ccn_embed.size(1), 0,: ] ) #We encode the first tok_embed.size(0) positions

class PositionalEncoding2D( nn.Module ):

  def __init__( self, d_model: int=256, dropout: float=0.1, height: int=10, width: int=10 ):
    super(PositionalEncoding2D, self).__init__()
    """
    :param d_model: dimension of the model
    :param height: height of the positions
    :param width: width of the positions
    :return: d_model*height*width position matrix
    """
    if d_model % 4 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                         "odd dimension (got dim={:d})".format(d_model))
    self.pe = torch.zeros(d_model, height, width).to(device)
    # Each dimension use half of d_model
    d_model = int(d_model / 2)
    div_term = torch.exp(torch.arange(0., d_model, 2) *
                         -(math.log(10000.0) / d_model))
    pos_w = torch.arange(0., width).unsqueeze(1)
    pos_h = torch.arange(0., height).unsqueeze(1)
    self.pe[0:d_model:2, :, :] = torch.sin(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
    self.pe[1:d_model:2, :, :] = torch.cos(pos_w * div_term).transpose(0, 1).unsqueeze(1).repeat(1, height, 1)
    self.pe[d_model::2, :, :] = torch.sin(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
    self.pe[d_model + 1::2, :, :] = torch.cos(pos_h * div_term).transpose(0, 1).unsqueeze(2).repeat(1, 1, width)
    self.dropout = nn.Dropout(dropout)
    self.register_buffer("pos_enc", self.pe)

  def forward(self, ccn_embed: Tensor):
    return self.dropout( ccn_embed + self.pe) #We encode the first tok_embed.size(0) positions


###################
# Input embedding #
###################
#1st approach: try passing a CCN output as an imput embedding, (ex: 400x180)
class SampleEmbedding( nn.Module ):

  def __init__(self, d_emb: int=256):
      super(SampleEmbedding, self).__init__()
      self.ccn = model_ft

  def forward(self, blocks: Tensor):
    return torch.utils.checkpoint.checkpoint_sequential(self.ccn, 5, blocks).to(device)

##################
# Token Embedder #
##################
class TokenEmbedding(nn.Module):

  def __init__(self, vocab_size: int, d_model: int):
    super(TokenEmbedding, self).__init__()
    self.embedder = nn.Embedding(vocab_size, d_model)
    self.emb_size = d_model

  def forward(self, tokens: Tensor):
    return (self.embedder( tokens.long() ) * math.sqrt(self.emb_size)).to(device)

###########
# Masking #
###########
def generate_square_subsequent_mask(sz):
    mask = (torch.tril(torch.ones((sz, sz))) == 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_pad_mask( matrix: torch.tensor, pad_token: int) -> torch.tensor:
    # If matrix = [1,2,3,0,0,0] where pad_token=0, the result mask is
    # [False, False, False, True, True, True]
    return (matrix == pad_token)

############
# Accuracy #
############
def batch_accuracy_calculate( pred: Tensor, y: Tensor):

  corr = 0
  tot = 0
  for p, label in zip(pred[:] ,y[:]):


    for tok_p, tok_l in zip(p[:], label[:]):


      if tok_p.item() != 3 and tok_l.item() != 3:
        corr += (tok_p == tok_l).type(torch.int).sum().item()
        tot += 1


      else:
        return [corr, tot]

  return [corr, tot]


In [ ]:
##############
# Tranformer #
##############
class TransformerModel(nn.Module):

  #ntokens referes to the latex vocabulary
  def __init__(self, ntokens: int, d_mod: int, d_hid: int ,n_head: int=8, dropout: float=0.5):
      super().__init__()
      self.model_type = "Transformer"
      self.nhead = n_head
      self.d_model = d_mod
      self.src_embedding = SampleEmbedding(d_mod)
      self.pos_encod = PositionalEncoding2D(d_mod, dropout, 6, 6)
      self.tgt_encod = PositionalEncoding(d_mod, dropout, maxlen=20)
      self.tgt_embed = nn.Embedding(n_tok,d_mod)
      self.flat = nn.Flatten(start_dim=2)
      encoder_layer = nn.TransformerEncoderLayer( d_mod, n_head, d_hid, batch_first=True )
      self.transformer_encoder = nn.TransformerEncoder(encoder_layer, n_head)
      decoder_layer = nn.TransformerDecoderLayer( d_mod, n_head, dropout=dropout, batch_first=True)
      self.transformer_decoder = nn.TransformerDecoder(decoder_layer, n_head )
      self.lin_soft_stack = nn.Sequential(
          nn.Linear(d_mod, ntokens),
          nn.Softmax(dim=1)
      )
      #self.init_weights()

  """def init_weights(self) -> None:
    initrange = 0.1
    self.embedding.weight.data.uniform_(-initrange, initrange)
    self.lin.bias.data.zero_()
    self.lin.weight.data.uniform_(-initrange, initrange)"""

  def forward(self, src: Tensor, tgt: Tensor, tgt_mask: Tensor, pad_mask: Tensor) -> Tensor:
    """
    Args:
       src: Tensor, shape [seq_len, batch_size]

    Returns:
       output Tensor of shape [seq_len, batch_size, ntoken]
    """
    src_emb = self.src_embedding(src) * math.sqrt(self.d_model)

    tgt = self.tgt_embed(tgt) * math.sqrt(self.d_model)
    src_emb = self.pos_encod(src_emb)
    tgt = self.tgt_encod(tgt)
    src_emb = self.flat(src_emb)
    #print(src_emb.size())
    memory = self.transformer_encoder(src_emb.permute(0,2,1))
    deco = self.transformer_decoder(tgt, memory, tgt_mask=tgt_mask, tgt_key_padding_mask=pad_mask)
    output = self.lin_soft_stack(deco)
    return output

In [ ]:
model = TransformerModel(n_tok, 256,1024).to(device)
#model = SampleEmbedding(512)
model = torch.nn.DataParallel(model)
"""for batch, (X, y, paths) in enumerate(test_dataloader):
    X, y = X.to(device), y.to(device)
    pred = model(X)

print(pred.size())"""

'for batch, (X, y, paths) in enumerate(test_dataloader):\n    X, y = X.to(device), y.to(device)\n    pred = model(X)\n\nprint(pred.size())'

In [ ]:
#Define train and test loop
def train_loop(dataloader, model, loss_fn, optimizer):
  model.train()
  size = 8300*20
  for batch, (X, y, paths) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    tgt_mask = generate_square_subsequent_mask(y.size(1))
    pad_mask = create_pad_mask(y, 3)
    #Compute prediction and loss
    pred = model(X, y, tgt_mask, pad_mask)
    #print(pred.size())
    loss = loss_fn(pred.permute(0,2,1), y)

    #Backpropagation
    optimizer.zero_grad()
   # print("Sono qui ", batch)
    loss.backward()
    #print("E ora qui", batch)
    optimizer.step()

    if batch % 10 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"Funzione di loss: {loss:>7f} [{current:>5d}/8300]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = 0
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
      for X, y, paths in dataloader:
        X, y = X.to(device), y.to(device)
        tgt_mask = generate_square_subsequent_mask(y.size(1))
        pad_mask = create_pad_mask(y, 3)
        #Compute prediction and loss
        pred = model(X, y, tgt_mask, pad_mask)
        loss = loss_fn(pred.permute(0,2,1), y)
        test_loss += loss_fn(pred.permute(0,2,1), y).item()
        acc_pack = batch_accuracy_calculate(pred.argmax(2), y) #(pred.argmax(2) == y).type(torch.int).sum().item()
        correct += acc_pack[0]
        size += acc_pack[1]

    test_loss /= num_batches
    correct /= size
    print(f"Errore test: \n Accuratezza: {(100*correct):>0.1f}%, Media loss: {test_loss:>8f} \n")

In [ ]:
#Definiamo la loss function e il metodo di minimizzazione
learning_rate = 1e-1
momentum = 0.9
batch_size = 64

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = momentum)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.98), eps=1e-9)
#train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
#test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

#Definire i tgt ovvero gli output

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

torch.save(model.state_dict(), 'model_weights.pth')
torch.save(model, 'model.pth')

Epoch 1
-------------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Funzione di loss: 4.606185 [    0/8300]
Funzione di loss: 4.603224 [  640/8300]
Funzione di loss: 4.603198 [ 1280/8300]
Funzione di loss: 4.599563 [ 1920/8300]
Funzione di loss: 4.584363 [ 2560/8300]
Funzione di loss: 4.548936 [ 3200/8300]
Funzione di loss: 4.540801 [ 3840/8300]
Funzione di loss: 4.527478 [ 4480/8300]
Funzione di loss: 4.509054 [ 5120/8300]
Funzione di loss: 4.458724 [ 5760/8300]
Funzione di loss: 4.405024 [ 6400/8300]
Funzione di loss: 4.391484 [ 7040/8300]
Funzione di loss: 4.351495 [ 7680/8300]
Errore test: 
 Accuratezza: 56.5%, Media loss: 4.284578 

Epoch 2
-------------------------------
Funzione di loss: 4.303141 [    0/8300]
Funzione di loss: 4.319856 [  640/8300]
Funzione di loss: 4.272949 [ 1280/8300]
Funzione di loss: 4.252143 [ 1920/8300]
Funzione di loss: 4.243358 [ 2560/8300]
Funzione di loss: 4.186568 [ 3200/8300]
Funzione di loss: 4.191045 [ 3840/8300]
Funzione di loss: 4.127498 [ 4480/8300]
Funzione di loss: 4.090259 [ 5120/8300]
Funzione di loss: 4.12

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')
torch.save(model, 'model.pth')

In [ ]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.
